In [1]:
#第十章：数据聚合与分组运算
#学习到使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象
#计算分组的概率统计，比如数量、平均值或标准差，或是用户定义的函数
#应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
#计算透视表或交叉表
#执行分位数分析以及其它统计分组分析
#笔记：对时间序列数据的聚合（groupby的特殊用法之一）也称重采样（resampling）

In [3]:
#10.1：GroupBy机制：split-apply-combine（拆分-应用-合并）
#分组键类型：
#1.list或array，其长度与待分组的轴一样
#2.表示DataFrame某个列名的值
#3.dict或Series，给出待分组轴上的值与分组名之间的对应关系
#4.函数，用于处理轴索引或索引中的各个标签
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                           'key2' : ['one', 'two', 'one', 'two', 'one'],
                           'data1' : np.random.randn(5),
                           'data2' : np.random.randn(5)})
df                     

,key1,key2,data1,data2
0,a,one,0.459712,-0.606303
1,a,two,1.301274,-0.129096
2,b,one,0.527394,-0.999252
3,b,two,0.709313,0.719762
4,a,one,1.271814,-0.129394


In [5]:
#对key1进行分组，并计算data1列的平均值：这里访问data1，并根据key1调用groupby：
grouped = df['data1'].groupby(df['key1'])
grouped
#grouped是一个GroupBy对象，其只是含有分组键df['key1']的中间值而已，但该对象已经含有接下来进行个分组运算
#的一切信息

In [6]:
grouped.mean()

key1
a    1.010934
b    0.618354
Name: data1, dtype: float64

In [10]:
#如果传入多个数组的列表，其结果为：
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means
#这里得到的是一个层次化索引（由唯一的键对组成）

key1  key2
a     one     0.865763
      two     1.301274
b     one     0.527394
      two     0.709313
Name: data1, dtype: float64

In [11]:
means.unstack()

key2,one,two
key1,,
a,0.865763,1.301274
b,0.527394,0.709313


In [12]:
#下例中的分组键均为Series，实际上，分组键可以是任何长度适当的数组：
states = np.array(['Ohio', 'California', 'California', 'Ohio','Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states,years]).mean()

California  2005    1.301274
            2006    0.527394
Ohio        2005    0.584513
            2006    1.271814
Name: data1, dtype: float64

In [13]:
#通常分组信息就位于相同的要处理的DataFrame中，可将列名（可以是字符串、数字或其他Python对象）用作分组键：
df.groupby(['key1']).mean()
#由于df['key2']不是数值数据，在结果中被排除

,data1,data2
key1,,
a,1.010934,-0.288264
b,0.618354,-0.139745


In [15]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.865763 -0.367848
     two   1.301274 -0.129096
b    one   0.527394 -0.999252
     two   0.709313  0.719762

In [16]:
#groupby的size方法，可返回一个含有分组大小的Series：
df.groupby(['key1','key2']).size()
#注意：任何分组关键词中的缺失值，都会被从结果中除去

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [17]:
#对分组进行迭代
#groupby支持迭代，可产生一组二元元组（由分组名和数据块组成）：
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.459712 -0.606303
1    a  two  1.301274 -0.129096
4    a  one  1.271814 -0.129394
b
  key1 key2     data1     data2
2    b  one  0.527394 -0.999252
3    b  two  0.709313  0.719762


In [18]:
#对于多重键的情况，元组的第一个元素将会是由键值组成的元组：
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.459712 -0.606303
4    a  one  1.271814 -0.129394
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.301274 -0.129096
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.527394 -0.999252
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.709313  0.719762


In [19]:
#可以对这些数据片段做成一个dict：
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.527394,-0.999252
3,b,two,0.709313,0.719762


In [22]:
#groupby默认是在axis=0上进行分组的，通过设置可在任意轴上进行分组:使用dtype对列进行分组：
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [26]:
grouped = df.groupby(df.dtypes,axis=1)

In [27]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.459712 -0.606303
1  1.301274 -0.129096
2  0.527394 -0.999252
3  0.709313  0.719762
4  1.271814 -0.129394
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [ ]:
#选取一列或列的子集：
#df.groupby('key1')['data1']  ==  df['data1'].groupby(df['key1'])
#df.groupby('key1')['data2']  ==  df['data1'].groupby(df['key1'])

In [29]:
#只需要计算data2列的平均值并以DataFrame形式得到结果：
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one    -0.367848
      two    -0.129096
b     one    -0.999252
      two     0.719762
Name: data2, dtype: float64

In [30]:
#通过字典或Series进行分组，除数组以外，分组信息还可以其他形式存在：
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan #add a few NA values
people

,a,b,c,d,e
Joe,1.268905,-0.059632,-0.580032,-1.258891,-0.535650
Steve,0.162455,-0.825444,-0.248125,1.703362,-0.623499
Wes,1.433919,NaN,NaN,-0.519533,0.407078
Jim,-1.577021,0.841543,-1.817199,-1.136116,0.267348
Travis,0.754226,1.296773,0.944520,-2.137418,0.499434


In [36]:
#假设已知列的分组关系，并根据分组计算列的和：
mapping = {'a':'red','b':'red','c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
#可将这个dict传给groupby，来构造数组，现在可以直接传递dict：
by_column = people.groupby(mapping,axis = 1)
by_column.mean()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [34]:
#Series也有同样的功能，可看作一个固定大小的映射：
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [35]:
people.groupby(map_series,axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [38]:
#默认索引为行，计算行名的len（即以行名称长度为分组信息）
people.groupby(len).sum()

,a,b,c,d,e
3,1.125803,0.781910,-2.397231,-2.914541,0.138776
5,0.162455,-0.825444,-0.248125,1.703362,-0.623499
6,0.754226,1.296773,0.944520,-2.137418,0.499434


In [39]:
#将函数跟数组、列表、字典、Series混合使用，因为任何东西在内部都会被转换为数组：
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  1.268905 -0.059632 -0.580032 -1.258891 -0.535650
  two -1.577021  0.841543 -1.817199 -1.136116  0.267348
5 one  0.162455 -0.825444 -0.248125  1.703362 -0.623499
6 two  0.754226  1.296773  0.944520 -2.137418  0.499434

In [43]:
#根据索引级别分组
#层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                    names = ['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.502961  0.701840  1.063947  0.960162 -0.868838
1     -1.867798 -1.127718 -0.863923 -0.266460  0.052594
2     -1.187283 -0.278617 -0.373745  1.367234 -0.150379
3     -0.929098 -1.387462  0.232646  0.597875 -1.194550

In [44]:
#根据级别分组，使用level关键字传递级别序号或名字：
hier_df.groupby(level ='cty',axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [45]:
#数据聚合
#聚合指任何能够从数组产生标量值的数据转换过程
df

,key1,key2,data1,data2
0,a,one,0.459712,-0.606303
1,a,two,1.301274,-0.129096
2,b,one,0.527394,-0.999252
3,b,two,0.709313,0.719762
4,a,one,1.271814,-0.129394


In [46]:
grouped = df.groupby('key1')

In [47]:
grouped['data1'].quantile(0.9)

key1
a    1.295382
b    0.691121
Name: data1, dtype: float64

In [48]:
grouped['data1'].mean()

key1
a    1.010934
b    0.618354
Name: data1, dtype: float64

In [49]:
#使用自己定义的聚合函数，只需将其出传入aggregate或agg方法即可：
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.841562,0.477206
b,0.181918,1.719014


In [50]:
#有些方法（如describe）也是可以用的，即使其并非聚合运算：
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.010934  0.477599  0.459712  0.865763  1.271814  1.286544   
b      2.0  0.618354  0.128636  0.527394  0.572874  0.618354  0.663833   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.301274   3.0 -0.288264  0.275429 -0.606303 -0.367848 -0.129394   
b     0.709313   2.0 -0.139745  1.215527 -0.999252 -0.569499 -0.139745   

                          
           75%       max  
key1                      
a    -0.129245 -0.129096  
b     0.290008  0.719762

In [53]:
#面向列的多函数应用
tips = pd.read_csv(r'examples\tips.csv')
tips.head(10)

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
5,25.29,4.71,No,Sun,Dinner,4
6,8.77,2.00,No,Sun,Dinner,2
7,26.88,3.12,No,Sun,Dinner,4
8,15.04,1.96,No,Sun,Dinner,2
9,14.78,3.23,No,Sun,Dinner,2


In [54]:
#增加总帐单的小费百分比：
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head(10)

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [55]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [57]:
#对不同的列使用不同的聚合函数：
grouped_pct = tips.groupby(['smoker','day'])
grouped_pct.agg('mean') #使用自定义

total_bill       tip      size   tip_pct
smoker day                                           
No     Fri    18.420000  2.812500  2.250000  0.151650
       Sat    19.661778  3.102889  2.555556  0.158048
       Sun    20.506667  3.167895  2.929825  0.160113
       Thur   17.113111  2.673778  2.488889  0.160298
Yes    Fri    16.813333  2.714000  2.066667  0.174783
       Sat    21.276667  2.875476  2.476190  0.147906
       Sun    24.120000  3.516842  2.578947  0.187250
       Thur   19.190588  3.030000  2.352941  0.163863

In [60]:
#如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名：
grouped_pct.agg(['mean','std',peak_to_peak])

total_bill                               tip            \
                  mean        std peak_to_peak      mean       std   
smoker day                                                           
No     Fri   18.420000   5.059282        10.29  2.812500  0.898494   
       Sat   19.661778   8.939181        41.08  3.102889  1.642088   
       Sun   20.506667   8.130189        39.40  3.167895  1.224785   
       Thur  17.113111   7.721728        33.68  2.673778  1.282964   
Yes    Fri   16.813333   9.086388        34.42  2.714000  1.077668   
       Sat   21.276667  10.069138        47.74  2.875476  1.630580   
       Sun   24.120000  10.442511        38.10  3.516842  1.261151   
       Thur  19.190588   8.355149        32.77  3.030000  1.113491   

                              size                          tip_pct            \
            peak_to_peak      mean       std peak_to_peak      mean       std   
smoker day                                                                      
No     Fri          2.00  2.250000  0.500000            1  0.151650  0.028123   
       Sat          8.00  2.555556  0.784960            3  0.158048  0.039767   
       Sun          4.99  2.929825  1.032674            4  0.160113  0.042347   
       Thur         5.45  2.488889  1.179796            5  0.160298  0.038774   
Yes    Fri          3.73  2.066667  0.593617            3  0.174783  0.051293   
       Sat          9.00  2.476190  0.862161            4  0.147906  0.061375   
       Sun          5.00  2.578947  0.901591            3  0.187250  0.154134   
       Thur         3.00  2.352941  0.701888            2  0.163863  0.039389   

                          
            peak_to_peak  
smoker day                
No     Fri      0.067349  
       Sat      0.235193  
       Sun      0.193226  
       Thur     0.193350  
Yes    Fri      0.159925  
       Sat      0.290095  
       Sun      0.644685  
       Thur     0.151240

In [61]:
#并非一定接受groupby自动给的列名，特别是lambda函数，则传递一个由（name,function)元组组成的列表，则各元组的第一个
#元素就会被用作DataFrame的列名（可将这种二元元组列表看作一个有序映射）
grouped_pct.agg([('foo','mean'),('bar',np.std)])

total_bill                  tip                size            \
                   foo        bar       foo       bar       foo       bar   
smoker day                                                                  
No     Fri   18.420000   5.059282  2.812500  0.898494  2.250000  0.500000   
       Sat   19.661778   8.939181  3.102889  1.642088  2.555556  0.784960   
       Sun   20.506667   8.130189  3.167895  1.224785  2.929825  1.032674   
       Thur  17.113111   7.721728  2.673778  1.282964  2.488889  1.179796   
Yes    Fri   16.813333   9.086388  2.714000  1.077668  2.066667  0.593617   
       Sat   21.276667  10.069138  2.875476  1.630580  2.476190  0.862161   
       Sun   24.120000  10.442511  3.516842  1.261151  2.578947  0.901591   
       Thur  19.190588   8.355149  3.030000  1.113491  2.352941  0.701888   

              tip_pct            
                  foo       bar  
smoker day                       
No     Fri   0.151650  0.028123  
       Sat   0.158048  0.039767  
       Sun   0.160113  0.042347  
       Thur  0.160298  0.038774  
Yes    Fri   0.174783  0.051293  
       Sat   0.147906  0.061375  
       Sun   0.187250  0.154134  
       Thur  0.163863  0.039389

In [66]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
smoker day                                                          
No     Fri        4  0.151650  0.187735          4  18.420000  22.75
       Sat       45  0.158048  0.291990         45  19.661778  48.33
       Sun       57  0.160113  0.252672         57  20.506667  48.17
       Thur      45  0.160298  0.266312         45  17.113111  41.19
Yes    Fri       15  0.174783  0.263480         15  16.813333  40.17
       Sat       42  0.147906  0.325733         42  21.276667  50.81
       Sun       19  0.187250  0.710345         19  24.120000  45.35
       Thur      17  0.163863  0.241255         17  19.190588  43.11

In [67]:
result['tip_pct']

count      mean       max
smoker day                            
No     Fri       4  0.151650  0.187735
       Sat      45  0.158048  0.291990
       Sun      57  0.160113  0.252672
       Thur     45  0.160298  0.266312
Yes    Fri      15  0.174783  0.263480
       Sat      42  0.147906  0.325733
       Sun      19  0.187250  0.710345
       Thur     17  0.163863  0.241255

In [68]:
#传入自定义名称的一组元组：
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
smoker day                                                  
No     Fri      0.151650   0.000791    18.420000   25.596333
       Sat      0.158048   0.001581    19.661778   79.908965
       Sun      0.160113   0.001793    20.506667   66.099980
       Thur     0.160298   0.001503    17.113111   59.625081
Yes    Fri      0.174783   0.002631    16.813333   82.562438
       Sat      0.147906   0.003767    21.276667  101.387535
       Sun      0.187250   0.023757    24.120000  109.046044
       Thur     0.163863   0.001551    19.190588   69.808518

In [71]:
#对一个列或不同的列使用不同的函数：
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
smoker day              
No     Fri    3.50     9
       Sat    9.00   115
       Sun    6.00   167
       Thur   6.70   112
Yes    Fri    4.73    31
       Sat   10.00   104
       Sun    6.50    49
       Thur   5.00    40

In [72]:
grouped.agg({'tip_pct':['min','max', 'mean', 'std'],
            'size':'sum'})
#只有将多个函数应用到至少一列时，DataFrame才会拥有层次化的列

tip_pct                               size
                  min       max      mean       std  sum
smoker day                                              
No     Fri   0.120385  0.187735  0.151650  0.028123    9
       Sat   0.056797  0.291990  0.158048  0.039767  115
       Sun   0.059447  0.252672  0.160113  0.042347  167
       Thur  0.072961  0.266312  0.160298  0.038774  112
Yes    Fri   0.103555  0.263480  0.174783  0.051293   31
       Sat   0.035638  0.325733  0.147906  0.061375  104
       Sun   0.065660  0.710345  0.187250  0.154134   49
       Thur  0.090014  0.241255  0.163863  0.039389   40

In [74]:
#以“没有行索引”的形式返回聚合数据
tips.groupby(['day','smoker'],as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [75]:
tips.groupby(['day','smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [76]:
#10.3 apply:一般性的“split-apply-combine"
#apply会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起
#根据分组选取最高的5个tip_pct值;首先，编写一个选取指定列具有最大值的行的函数：
def top(df,n=5,column = 'tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [77]:
top(tips)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [78]:
#对smoker分组并用该函数调用apply：
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [79]:
#传给apply的函数能够接受其他参数或关键字，则可以将这些内容放在函数名后面一并传入：
tips.groupby(['smoker','day']).apply(top,n=1,column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [81]:
#调用groupby对象上的describe
result = tips.groupby('smoker')['tip_pct'].describe()
result
#在groupby中，调用诸如describe之类的方法时，实际上只是应用下面两条代码的快捷方式而已;如：
#f = lambda x:x.describe()
#grouped.apply(f)

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [82]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [84]:
#禁止分组键：group_keys = False传入groupby即可禁止该效果：
tips.groupby('smoker',group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [89]:
#分位数和桶分析
#结合pandas中的拆分工具--如cut和qcut（根据指定面元或样本分位数），groupby可轻松的实现对数据集的桶（bucket）
#或分位数（quantile）分析了：
frame = pd.DataFrame({'data1':np.random.randn(1000),
                    'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)
quartiles[:10]

0    (-1.531, 0.0153]
1     (0.0153, 1.562]
2    (-1.531, 0.0153]
3     (0.0153, 1.562]
4    (-3.084, -1.531]
5     (0.0153, 1.562]
6      (1.562, 3.109]
7     (0.0153, 1.562]
8    (-1.531, 0.0153]
9     (0.0153, 1.562]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.084, -1.531] < (-1.531, 0.0153] < (0.0153, 1.562] < (1.562, 3.109]]

In [92]:
#由cut返回的Categorical对象可直接传递到groupby；因此，可对data2列做一些如下统计：
def get_stats(group):
    return{'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.084, -1.531]",56.0,1.925019,0.034252,-1.903579
"(-1.531, 0.0153]",419.0,2.917733,0.028424,-2.818325
"(0.0153, 1.562]",471.0,2.843836,0.027630,-2.903385
"(1.562, 3.109]",54.0,2.073181,-0.008087,-2.345422


In [93]:
grouped.apply(get_stats)

data1                  
(-3.084, -1.531]  count     56.000000
                  max        1.925019
                  mean       0.034252
                  min       -1.903579
(-1.531, 0.0153]  count    419.000000
                  max        2.917733
                  mean       0.028424
                  min       -2.818325
(0.0153, 1.562]   count    471.000000
                  max        2.843836
                  mean       0.027630
                  min       -2.903385
(1.562, 3.109]    count     54.000000
                  max        2.073181
                  mean      -0.008087
                  min       -2.345422
Name: data2, dtype: float64

In [94]:
#要根据样本分位数得到大小相等的桶，使用qcut即可，传入：labels=False即可获得分位数的编号
#Return auantile numbers
grouping = pd.qcut(frame.data1,10,labels = False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.925019,-0.044384,-2.763451
1,100.0,2.515374,-0.016172,-2.818325
2,100.0,1.993467,-0.072181,-2.356506
3,100.0,2.511574,0.188930,-2.600287
4,100.0,2.917733,0.119924,-2.171010
5,100.0,2.366653,-0.064954,-2.629481
6,100.0,2.390813,0.075893,-2.459956
7,100.0,2.634888,0.014524,-2.808126
8,100.0,2.442548,0.042422,-2.903385


In [96]:
#示例：用特定于分组的值填充缺失值
#缺失值的处理，有时使用dropna处理，有时候希望用一个固定值或由数据集本身所衍生出来的值来填充NA值，这时就可以使用
#fillna这个工具了；如使用平均值填充NA值
s = pd.Series(np.random.randn(6))
s[::2] =np.nan
s

0         NaN
1    0.911013
2         NaN
3    0.985493
4         NaN
5    0.324337
dtype: float64

In [97]:
s.fillna(s.mean())

0    0.740281
1    0.911013
2    0.740281
3    0.985493
4    0.740281
5    0.324337
dtype: float64

In [98]:
#假设需要对不同的分组填充不同的值
#将数据分组，并使用apply和一个能够对割数据块调用fillna的函数即可：
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East']*4 + ['West']*4
data = pd.Series(np.random.randn(8),index = states)
data

Ohio         -0.304702
New York      1.025153
Vermont      -0.974411
Florida       1.977131
Oregon       -1.205008
Nevada       -1.633862
California   -0.548135
Idaho        -0.028357
dtype: float64

In [100]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.304702
New York      1.025153
Vermont            NaN
Florida       1.977131
Oregon       -1.205008
Nevada             NaN
California   -0.548135
Idaho              NaN
dtype: float64

In [101]:
data.groupby(group_key).mean()

East    0.899194
West   -0.876572
dtype: float64

In [102]:
#用平均值去填充不同分组的NA值：
fill_mean = lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.304702
New York      1.025153
Vermont       0.899194
Florida       1.977131
Oregon       -1.205008
Nevada       -0.876572
California   -0.548135
Idaho        -0.876572
dtype: float64

In [103]:
#可以在代码中预定义各组的填充值，由于分组具有一个name属性，则：
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.304702
New York      1.025153
Vermont       0.500000
Florida       1.977131
Oregon       -1.205008
Nevada       -1.000000
California   -0.548135
Idaho        -1.000000
dtype: float64

In [105]:
#示例：随机采样和排序
#假设要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡洛模拟（Monte Carlo simulation）或其他分析工作：
#“抽取”的方式有很多，这里使用的方法是对Series使用sample方法：
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [109]:
#以上含有一个长度为52的Series，其索引包括牌名，值则是21点或其他游戏中用于计分的点数：
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [112]:
#从整副拍中抽出5张：
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

4C      4
8C      8
JD     10
AD      1
10D    10
dtype: int64

In [113]:
#根据每个花色随机抽取两张牌，花色是牌名的最后一个字符，则可根据此进行分组，并使用apply：
get_suit = lambda card:card[-1]  #last letter is suit
deck.groupby(get_suit).apply(draw,n=2)

C  7C     7
   KC    10
D  KD    10
   3D     3
H  8H     8
   9H     9
S  2S     2
   4S     4
dtype: int64

In [115]:
#上式可写成
deck.groupby(get_suit,group_keys = False).apply(draw,n=2)

AC      1
QC     10
7D      7
KD     10
AH      1
7H      7
JS     10
10S    10
dtype: int64

In [124]:
#示例：分组加权平均数和相关系数
#根据groupby的“split-apply-combine”范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)}) 
df

,category,data,weights
0,a,-0.370731,0.063092
1,a,-0.690872,0.638502
2,a,-0.518894,0.544517
3,a,-0.095390,0.109977
4,b,-0.385137,0.740001
5,b,0.302679,0.950519
6,b,0.146182,0.792496
7,b,1.226792,0.927199


In [126]:
#可以利用category计算分组加权平均数：
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'],weights = g['weights'])
grouped.apply(get_wavg)

category
a   -0.558629
b    0.368314
dtype: float64

In [127]:
#另一个例子：含有几只股票和标准普尔500指数（符号SPX）的收盘价：
close_px = pd.read_csv('examples/stock_px_2.csv',parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [129]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [131]:
#计算一个由日收益率（通过百分数变化计算）与SPX之间年度相关系数组成的DataFrame：
#方法：先创建一个函数，用它计算每列和SPX列的成对相关系数：
spx_corr = lambda x:x.corrwith(x['SPX'])
#接下来使用pct_change计算close_px的百分比变化：
rets = close_px.pct_change().dropna()
#最后用年对百分比变化进行分组，可以用一个一行的函数，从每行的标签返回每个datetime标签的year属性：
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [132]:
#可以计算列与列之间的相关系数;这里，使用Apple和Microsoft的年相关系数：
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [135]:
#组级别的线性回归
#利用statsmodels计量经济学库的regress函数对各数据执行普通最小二乘法（Ordinary Least Squares，OLS）回归：
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y,X).fit()
    return result.params
#为了按年计算AAPL对SPX收益率的线性回归，执行：
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [136]:
#透视表（pivot table）和交叉表（crosstab）
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [137]:
#只想聚合tip_pct和size，而且想根据time进行分组，将smoker放在列上，把day放在行上：
tips.pivot_table(['tip_pct','size'],index = ['time','day'],columns = 'smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [139]:
#还可以对这个表作进一步处理，传入margins = True添加分项小计，这将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计：
tips.pivot_table(['tip_pct','size'],index = ['time','day'],columns = 'smoker',margins = True)
#All值为平均值：不单独考虑烟民与非烟民（All列），不单独考虑行分组两个级别中的任何单项（All行）

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [140]:
#要使用其他的聚合函数，将其传给aggfunc即可；例如，使用count或len可得到有关分组大小的交叉表（计数或频率）
tips.pivot_table('tip_pct',index =['time','smoker'],columns = 'day',aggfunc = len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [143]:
#如果是一个空的组合（即NA），可设置一个fill_value:
tips.pivot_table('tip_pct',index = ['time','size','smoker'],
                columns = 'day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [151]:
#交叉表（cross-tabulation,简称crosstab)是一种用于计算分组频率的特殊透视表：
data = pd.DataFrame({'Sample':list(range(1,11)),
                    'Nationality':['USA','Japan','USA','Japan','Japan','Japan','USA','USA','Japan','USA'],
                    'Handedness':['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed',
                                  'Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']})
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [160]:
#根据国籍和用手习惯对数据进行统计汇总，可用pivot_table实现，但pandas.crosstab函数会更方便
pd.crosstab(data.Nationality,data.Handedness,margins = True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [168]:
grouped = data.groupby('Nationality').count()
grouped

,Sample,Handedness
Nationality,,
Japan,5,5
USA,5,5


In [169]:
#crosstab的前两个参数可以是数组或Series，或是数组列表：
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [ ]:
#10.5:总结：掌握pandas数据分组工具既有助于数据清理，也有助于建模或统计分析工作